<a href="https://colab.research.google.com/github/zawadif/colab-notebooks/blob/master/Related_Categories_Fila_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Related Categories Fila 👟

Here is the basic idea for this workflow: gathering relevant reccomendations to cross-link similar category pages. Here follows the list of steps:  

1. Gather data starting from the sitemap 
2. Parse all the text elements we need
3. Extract queries from Google Search Console
4. Run Semantic Search 
  - Extract top n matches (semantic similarity)
  - Re-rank results (additional business logic)
5. Prepare the output dataset


---


<table align="left">
  <td>
  <a href="https://wordlift.io">
    <img width=130px src="https://wordlift.io/wp-content/uploads/2018/07/logo-assets-510x287.png" />
    </a>
    </td>
    <td>
      by 
      <a href="https://wordlift.io/blog/en/entity/andrea-volpini">
        Andrea Volpini
      </a>
      MIT License
      <br/>
      <br/>
      <i>Last updated: <b>Jun 2nd, 2022</b></i>
  </td>
</table>
</br>
</br>

## 0. Install libraries

In [ ]:
%%capture
!pip install advertools
!pip install git+https://github.com/joshcarty/google-searchconsole
!pip install spacy
!pip install -U sentence-transformers

from tqdm import tqdm
import searchconsole
import advertools as adv
import pandas as pd
import numpy as np
import spacy
import torch
import re
import json


from sentence_transformers import SentenceTransformer, util
from itertools import islice

from IPython.display import HTML


## 1. Getting the list of URLs from the sitemap

We will focus on category pages. These can be detected by analyzing the URL structure. 

In [ ]:
fl_sitemap = adv.sitemap_to_df('https://www.fila.com/sitemap_0.xml', recursive=False)
fl_sitemap

,loc,lastmod,changefreq,priority,sitemap,sitemap_size_mb,download_date
0,https://www.fila.com/kids-logo-tee/22F77K.html,2022-04-06 14:54:49+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
1,https://www.fila.com/kids-logo-tee/52F77K.html,2022-04-06 14:54:55+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2,https://www.fila.com/kids-t-shirt-dress/73F358...,2022-04-06 14:54:53+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
3,https://www.fila.com/kids-logo-short/81F335.html,2022-06-02 17:19:51+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
4,https://www.fila.com/kids-mesh-shorts/81F363.html,2022-05-27 16:47:58+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
...,...,...,...,...,...,...,...
2769,https://www.fila.com/customer-service/returns,2014-03-26 17:23:11+00:00,daily,0.5,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2770,https://www.fila.com/customer-service/shipping,2020-02-14 21:42:15+00:00,daily,0.5,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2771,https://www.fila.com/customer-service/your-acc...,2014-03-26 17:23:11+00:00,daily,0.5,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2772,https://www.fila.com/us,2022-06-13 03:41:56+00:00,daily,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00


In [ ]:
# Adding here the list of keywords to catch non-category pages (for example every .html corresponds to product page)

exclude_list = ['.html', 'about-', 'category-by-gender', 'your-account', 'news', 'job', 'home-', 'search?', 'marketing-landing-pages', 'women-new',
                'customer-service', 'us', 'on/', 'plrolando', 'gift', 'press', 'pl-landing', '-faq', 'test', 'shops', 'anniversary', 'saint-jhn-',
                'women-featured-playfulpinks', 'men-featured-cement', 'tennis-event-', 'fila-style', 'shoes-women-filaray', 'black-history']
 
category_sitemap = fl_sitemap[fl_sitemap["loc"].str.contains('|'.join(exclude_list))==False]
category_sitemap

,loc,lastmod,changefreq,priority,sitemap,sitemap_size_mb,download_date
2517,https://www.fila.com/men-heritage-outerwear,2021-02-19 02:29:30+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2519,https://www.fila.com/men-featured,2021-12-22 17:47:49+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2520,https://www.fila.com/shoes-men,2022-02-22 20:38:13+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2521,https://www.fila.com/shoes-featured,2021-10-07 14:27:05+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2522,https://www.fila.com/men-featured-drop,2020-03-16 19:04:39+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
...,...,...,...,...,...,...,...
2731,https://www.fila.com/robin-montgomery,2022-03-02 19:34:56+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2732,https://www.fila.com/company-mens,2020-06-22 19:18:13+00:00,daily,0.5,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2733,https://www.fila.com/company-womens,2020-06-22 19:18:38+00:00,daily,0.5,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00
2736,https://www.fila.com/chunky-sneakers,2022-04-06 22:16:07+00:00,weekly,1.0,https://www.fila.com/sitemap_0.xml,0.584495,2022-06-13 08:08:36.099618+00:00


## 2. Extract text from category pages

We will now configure a few selectors using XPATH to extract intro text and breadcrumbs. Data from the crawler will be saved on a `.jl` file. 

In [ ]:
xpath_selectors = {
    'intro': '//*[(@id = "main")]', # intro text
    'breadcrumbs' : '//*[contains(concat( " ", @class, " " ), concat( " ", "category-title-secondary", " " ))]//text()' # nav breadcrumbs
}

adv.crawl(url_list=category_sitemap['loc'],
          output_file='fl_category_crawl.jl',
          xpath_selectors=xpath_selectors)

In [ ]:
crawl_df_all = pd.read_json('fl_category_crawl.jl', lines=True)
crawl_df_all.drop_duplicates(subset=['url','title'], inplace=True)
crawl_df = crawl_df_all[['url', 'title', 'meta_desc', 'intro', 'breadcrumbs']] # here are the fields we'll use
crawl_df

,url,title,meta_desc,intro,breadcrumbs
0,https://www.fila.com/women-featured,"FILA Women's Featured Shoes, Clothing, & Acces...","Shop Women's Featured Shoes, Clothing, & Acce...","<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
1,https://www.fila.com/men-featured,"FILA Men's Featured Shoes, Clothing, & Accesso...","Shop Men's Featured Shoes, Clothing, & Access...","<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
2,https://www.fila.com/men-slides,Men's Slides + Sport Sandals | FILA,Men's slides and sports sandals by FILA: thou...,"<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
3,https://www.fila.com/men-shoes,Men's Shoes | FILA,"Men's sneakers, slides, and tennis shoes: FIL...","<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
4,https://www.fila.com/shoes-men,Men's Shoes | FILA,"Shop our collection of Men's Shoes, sneakers,...","<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
...,...,...,...,...,...
87,https://www.fila.com/ohsu-womens,Women's 40% Off,Fila.com,"<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
88,https://www.fila.com/pickleball-women-sneakers,Women's Pickleball Shoes | FILA,Our Women's Pickleball Shoes are designed to ...,"<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
89,https://www.fila.com/robin-montgomery,Robin Montgomery in FILA Tennis Apparel | FILA,Tennis great Robin Montgomery sports FILA ten...,"<div id=""main"" role=""main"" class=""clearfix"">\n...",\n \n @@\n @@\n ...
90,https://www.fila.com/grant-hill-sprite,Grant Hill Shoes x Sprite | FILA,Grant Hill Shoes x Sprite - a refreshing coll...,NaN,NaN


### Getting title

In [ ]:
def get_data(item):
  try: 
    if item is not None:                            
      if item.__contains__('|'):
        return item.split('|')[0]
      else:
        return item
  except:
    return ""      

crawl_df['title'] = crawl_df['title'].apply(get_data)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


### Getting Intro


In [ ]:
crawl_df['intro']
crawl_df['intro'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r|<[^<]+?>"], value=[" "," "], regex=True, inplace=True)
crawl_df['intro'].dropna()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


0                                                   ...
1                                                   ...
2                                                   ...
3                                                   ...
4                                                   ...
                            ...                        
86                                                  ...
87                                                  ...
88                                                  ...
89                                                  ...
91                                                  ...
Name: intro, Length: 89, dtype: object

### Getting Breadcrumbs

This is a quick check on the breadcrumbs. We might use the resulting dataframe to filter out links that already exist on the page. 

In [ ]:
struct_df = (pd.DataFrame({
    'url': crawl_df['url'],
    'breadcrumbs': crawl_df['breadcrumbs'].str.split('@@')
    })
 .explode('breadcrumbs'))

struct_df['breadcrumbs'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r|Home|sale|FILA|% Off"], value=[None,None], regex=True, inplace=True)
struct_df.dropna(axis=0, how='any', thresh=None, inplace=True, subset=['breadcrumbs'])
struct_df

,url,breadcrumbs
0,https://www.fila.com/women-featured,Featured
1,https://www.fila.com/men-featured,Featured
2,https://www.fila.com/men-slides,Sandals & Slides
3,https://www.fila.com/men-shoes,Shoes
4,https://www.fila.com/shoes-men,Men's Shoes
...,...,...
84,https://www.fila.com/slip-resistant,Slip Resistant Shoes
85,https://www.fila.com/best-sellers,Best Sellers
86,https://www.fila.com/work-men,Men's Footwear
88,https://www.fila.com/pickleball-women-sneakers,Shoes


### Get final text

We will proceed with some text cleaning, we will use Spacy, do some replacement and remove a list of stop words. 

In [ ]:
def clean_text(_input, _stop_words):

  # Here is the NLP model for English
  nlp = spacy.load('en_core_web_sm')
 
  # Clean up characters
  txt = re.sub('[^a-zA-Z \n\.]', '', _input)

 
  # Crate a spaCy document
  doc = nlp(txt)

  # Remove stop words, punctuation, numbers, and short terms
  clean_txt = [word.lemma_.lower() for word in doc if word.is_stop ==
                  False and word.is_punct == False and word.lemma_.lower() 
                  not in excluded_words and word.like_num == False and len(word.lemma_) > 3]

  # More cleanup   
  replaced_txt = ' '.join(list(filter(str.strip, clean_txt)))
  final_text = re.sub(r"@@[\n\t]*", "", replaced_txt)

  return final_text



In [ ]:
# adding here the stop words and run a first test

excluded_words = ['buy', 'sale', 'save', 'gift', 'purchase', 'more', 'free', 'FILA', 'home', 'official', 'refine', 'price', 'low', 'high' 
                  'shipping', 'order', 'store', 'status', 'shop', 'online', 'return', 'new', 'nan', '% Off',  'result', 'filter']

clean_text("fila women featured shoes clothing accessories women featured shoes clothing accessory fila.com enjoy shipping fila women featured featured women style deserve attention refine result filter(0 featured disruptor 2color size price shoe style low collections disruptor filter results clear filters sort price low high price high low items //<!--/ -->//<!--/ -->//<!--/ exclusive woman disruptor wedge$108.00//<!--/ -->//<!--/ -->//<!--/ exclusive woman disruptor wedge$108.00//<!--/ -->//<!--/ -->//<!--/ exclusive woman disruptor wedge$108.00//<!--/ -->//<!--/ -->//<!--/ woman disruptor premium$68.00 items fila women featured", excluded_words)

'fila woman feature shoe clothing accessory woman feature shoe clothing accessory fila.com enjoy ship fila woman feature feature woman style deserve attention feature disruptor color size shoe style collection disruptor clear sort high high item exclusive woman disruptor wedge exclusive woman disruptor wedge exclusive woman disruptor wedge woman disruptor premium item fila woman feature'

In [ ]:
crawl_df['all_text'] = crawl_df[['title', 'meta_desc', 'intro', 'breadcrumbs']].astype(str).apply(lambda x: ' '.join(x), axis=1)

#Create new column
crawl_df['clean_text'] = ""

#Looping through DataFrame and Process Text (takes some time)
for i, text in enumerate(crawl_df['all_text']):
    new_text = clean_text(str(text), excluded_words)
    crawl_df.at[i, 'clean_text'] = new_text
    
print(crawl_df['clean_text'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0     fila womens featured shoes clothing accessory ...
1     fila mens featured shoes clothing accessory me...
2     mens slide sport sandal mens slide sport sanda...
3     mens shoes mens sneaker slide tennis shoe fila...
4     mens shoes collection mens shoes sneaker fila....
                            ...                        
87    womens fila.com fila oregon health science uni...
88    womens pickleball shoes womens pickleball shoe...
89    robin montgomery fila tennis apparel tennis gr...
90    grant hill shoes sprite grant hill shoes sprit...
91    womens fila.com fila small business womens wom...
Name: clean_text, Length: 92, dtype: object


Let's review an example of text for one of our categories. 

In [ ]:
print('title:', crawl_df['title'][2], '\n'
      'clean_text:', crawl_df['clean_text'][2])

title: Men's Slides + Sport Sandals  
clean_text: mens slide sport sandal mens slide sport sandal fila thoughtfully design expertly craft slide sandal fila shoes sandals slides mens slide sandal thoughtfully designed slide sandals sport sandals slip walk away shoes sandal slides color size shoe style slide collections disruptor drifter outdoor slide results clear filters sort high high items solslide lunar slide lunar slide lunar slide lunar slide mens drifter mens disruptor sandal mens disruptor sandal outdoor slide mens utility silde mens drifter mens drifter mens drifter mens drifter mens drifter mens atlas slide exclusive terry drifter slide exclusive terry drifter slide exclusive terry drifter slide exclusive terry drifter slide exclusive terry drifter slide arrival mens drifter mens solslide arrival mens drifter items shoe sandal slides


## 3. Get data from GSC

Use this block if you have access to **Google Search Console**. It will get the top queries for every URL. 



> 👋 We are not using here this block, you can use it if you have access to GSC.



### Setup (connecting to GSC API)


##### Authenticate with GSC 

In [ ]:
#upload client_id.json and credentials.json files

from google.colab import files

names = files.upload()

In [ ]:
filename=list(names.keys())[0]

In [ ]:
account = searchconsole.authenticate(client_config=filename, serialize='credentials.json', flow="console")

#### Data extraction

In [ ]:
def get_search_data_by_page(_domain, _days, _page=None):
  #creating the webproperty
  webproperty = account[_domain]

  if None not in (webproperty, _page):
  #if webproperty is not None and we receive the page filter
    query = webproperty.query.range(start='today', days=_days).dimension('page', 'query').filter('page', _page, 'equals') #here alternatively we can use 'contains'
    r = query.get()
    df = pd.DataFrame(r.rows)
    return df
  elif webproperty is not None and _page is None:
  #if webproperty is not None and we DO NOT receive the page filter  
    query = webproperty.query.range(start='today', days=_days).dimension('page').filter('page', '/category/', 'contains') # adding here a filter to get only /category/ urls 
    r = query.get()
    df = pd.DataFrame(r.rows)
    return df

  print("Web property doesn't exist, please select a valid one from this list")
  return None


###### Run a test with `get_search_data_by_page()`

We will test the GSC function with a page filter. 

In [ ]:
df = get_search_data_by_page('sc-domain:oakley.com', -120, 'https://www.example.com/en-us/category/sunglasses/sport')
df[:5]['query'].tolist()

##### Find top queries for each page

In [ ]:
def find_top_queries(_domain, _page, _days=-120, _top_n=5):
  # calling GSC
  df = get_search_data_by_page(_domain, _days, _page)
  # getting top queries
  l = df[:_top_n]['query'].tolist()

  # removing the stop words from the queries
  queries = [clean_text(query, excluded_words) for query in l] 
  
  # removing empty items and duplicates
  queries = list(filter(None, queries))
  unique = set(queries)

  # getting clicks
  clicks = df['clicks'].sum()

  return unique, clicks


###### Run a test with `find_top_queries(_domain, _page, _days=-120, _top_n=5)`

The function gets the domain and the page and produces a list of queries. Stop words are removed from queries. This is helpful for instance if you want to remove the name of the brand. 

In [ ]:
queries, clicks = find_top_queries('sc-domain:example.com', 'https://www.example.com/en-us/category/sale/sweatshirts')
queries, clicks

### Looping through the list of categories

Here is where we extract for all categories the list of queries. 

In [ ]:
DOMAIN = 'sc-domain:example.com'
DAYS = '-120'

queries = []
clicks = []

with tqdm(crawl_df["url"], colour='#007AFF', smoothing=0.3) as t:
  for value in t:
    t.set_description("Processing « %s »" % str(value))
    try:
      q, c = find_top_queries(DOMAIN, value)
    except: 
      q=None
      c=None
  
    queries.append(q)
    clicks.append(c)

crawl_df["queries"] = queries
crawl_df["clicks"] = clicks

### Creating the text blob

Now we combine the extracted text with the top queries into a list. 

In [ ]:
def listToString(s): 
    
    # initialize a string
    str1 = "" 
    # traverse in the string  
    for ele in s: 
        str1 += ele + " " 
    # return string  
    return str1 

def createTextandQuery(_list1):
  text_and_queries = []

  for i, ele in enumerate(_list1):
      if crawl_df["queries"].iloc[i]: 
        s = ele + listToString(crawl_df["queries"].iloc[i])
      else:
        s = ele  
      
      text_and_queries.append(s)

  return text_and_queries

# here we store the final blob
text_and_queries = createTextandQuery(crawl_df['clean_text'].tolist())


## 4. Computing similarity




### Export to csv

We can save here the final dataset to be indexed.

In [ ]:
fila_df = crawl_df[['url', 'title', 'clean_text']]
fila_df.to_csv('fila-com.csv')

#### Running a quick test

Here is a quick test where we'll evaluate the recommended links for the following terms:

```
['boots for men', 'cycling', 'sneakers', 'kids']
```



In [ ]:
pd.options.display.max_colwidth = 100
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = crawl_df['clean_text'].tolist() # without queries
#corpus = text_and_queries # with queries

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['boots for men', 'cycling', 'sneakers', 'kids']

# Find the closest n sentences of the corpus for each query sentence based on cosine similarity

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0] # Get the hits for the first query
    for hit in hits:
        print('url:', crawl_df.iloc[[hit['corpus_id']]]['url'], 
              'title:', crawl_df.iloc[[hit['corpus_id']]]['title'],
              #'queries:',crawl_df.iloc[[hit['corpus_id']]]['queries'], # uncomment both if you have GSC data
              #'clicks' : crawl_df.iloc[[hit['corpus_id']]]['clicks']) 
              'hit:', hit)


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]





Query: boots for men

Top 5 most similar sentences in corpus:
url: 118    https://www.fila.com/shoes-featured-boots
Name: url, dtype: object title: 118    Men's & Women's Boots + Winter Sneakers 
Name: title, dtype: object hit: {'corpus_id': 118, 'score': 0.5999026298522949}
url: 106    https://www.fila.com/women-boots
Name: url, dtype: object title: 106    Women's Boots 
Name: title, dtype: object hit: {'corpus_id': 106, 'score': 0.5791897773742676}
url: 110    https://www.fila.com/men-boots
Name: url, dtype: object title: 110    Men's Boots 
Name: title, dtype: object hit: {'corpus_id': 110, 'score': 0.5696355104446411}
url: 127    https://www.fila.com/shoes-featured-raytracer
Name: url, dtype: object title: 127    Ray Tracer Chunky Tennis Shoe 
Name: title, dtype: object hit: {'corpus_id': 127, 'score': 0.5514737367630005}
url: 4    https://www.fila.com/men-sneakers
Name: url, dtype: object title: 4    Men's Casual Sneakers + Athletic Shoes 
Name: title, dtype: object hit: {'cor

Batches:   0%|          | 0/1 [00:00<?, ?it/s]





Query: cycling

Top 5 most similar sentences in corpus:
url: 41    https://www.fila.com/womens-bike-shorts
Name: url, dtype: object title: 41    Women's Biker Shorts - PERFORMA 
Name: title, dtype: object hit: {'corpus_id': 41, 'score': 0.34504806995391846}
url: 16    https://www.fila.com/women-heritage-pants
Name: url, dtype: object title: 16    Women's Bike Shorts + Joggers 
Name: title, dtype: object hit: {'corpus_id': 16, 'score': 0.3008688986301422}
url: 78    https://www.fila.com/womens-activewear
Name: url, dtype: object title: 78    Women's Workout Clothes + Apparel - PERFORMA 
Name: title, dtype: object hit: {'corpus_id': 78, 'score': 0.2759683132171631}
url: 20    https://www.fila.com/women-heritage-leggings
Name: url, dtype: object title: 20    Women's Athletic Leggings + Tights 
Name: title, dtype: object hit: {'corpus_id': 20, 'score': 0.2139626294374466}
url: 29    https://www.fila.com/men-heritage-pants
Name: url, dtype: object title: 29    Men's Athletic Pants + Sho

Batches:   0%|          | 0/1 [00:00<?, ?it/s]





Query: sneakers

Top 5 most similar sentences in corpus:
url: 4    https://www.fila.com/men-sneakers
Name: url, dtype: object title: 4    Men's Casual Sneakers + Athletic Shoes 
Name: title, dtype: object hit: {'corpus_id': 4, 'score': 0.6605402231216431}
url: 143    https://www.fila.com/chunky-sneakers
Name: url, dtype: object title: 143    Chunky Sneakers & Dad Sneakers 
Name: title, dtype: object hit: {'corpus_id': 143, 'score': 0.6408480405807495}
url: 46    https://www.fila.com/shoes-men-raytracer
Name: url, dtype: object title: 46    FILA Ray Tracer Men's Chunky Sneakers 
Name: title, dtype: object hit: {'corpus_id': 46, 'score': 0.6406381130218506}
url: 35    https://www.fila.com/women-sneakers
Name: url, dtype: object title: 35    Women's Casual Sneakers 
Name: title, dtype: object hit: {'corpus_id': 35, 'score': 0.6291723251342773}
url: 81    https://www.fila.com/shoes-men-sneakers
Name: url, dtype: object title: 81    Men's Sneakers 
Name: title, dtype: object hit: {'corp

Batches:   0%|          | 0/1 [00:00<?, ?it/s]





Query: kids

Top 5 most similar sentences in corpus:
url: 98    https://www.fila.com/kids-heritage-bottoms
Name: url, dtype: object title: 98    Kids' Shorts and Joggers 
Name: title, dtype: object hit: {'corpus_id': 98, 'score': 0.41436538100242615}
url: 76    https://www.fila.com/kids-shoes-little
Name: url, dtype: object title: 76    Little Kids' Shoes & Sneakers 
Name: title, dtype: object hit: {'corpus_id': 76, 'score': 0.3987221419811249}
url: 75    https://www.fila.com/kids-shoes-big
Name: url, dtype: object title: 75    Big Kids' Shoes & Sneakers 
Name: title, dtype: object hit: {'corpus_id': 75, 'score': 0.374586820602417}
url: 71    https://www.fila.com/shoes-kids
Name: url, dtype: object title: 71    FILA Kids' Shoes  
Name: title, dtype: object hit: {'corpus_id': 71, 'score': 0.36998146772384644}
url: 49    https://www.fila.com/shoes-kids-mb
Name: url, dtype: object title: 49    MB Kids' Basketball Shoes 
Name: title, dtype: object hit: {'corpus_id': 49, 'score': 0.3668

### Analyzing the entire set of categories and building the final list of links

Here we have the flow that will run the procedure for all the categories. 

In [ ]:
rows_list = []
for i, row in enumerate(tqdm(crawl_df.itertuples(index=True, name='Pandas'), colour='#007AFF', smoothing=0.3)):
    title = row.title
    url = row.url
    
    query_embedding = embedder.encode(title, convert_to_tensor=True)

    
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=7) # Get up to 7 recommendations 
    hits = hits[0] # Get the hits for the first title
    
    limit = 5 # Limit recommendations to 5 
    
    for s, hit in enumerate(islice(hits, limit)):
        
        print('ref:', url)
        dict1 = {}  
        # Make sure we don't suggest the same category (url) or the homepage among the related links
        if crawl_df.iloc[[hit['corpus_id']]]['url'].item() not in [url, "https://www.fila.com"]: 
            
          print('recommended:', crawl_df.iloc[[hit['corpus_id']]]['url'].item())
          dict1.update({'url': str(crawl_df.iloc[[hit['corpus_id']]]['url'].item()),
                 'title' : str(crawl_df.iloc[[hit['corpus_id']]]['title'].item())})  
                 #'queries' : crawl_df.iloc[[hit['corpus_id']]]['queries'], # uncomment both if you have GSC data
                 #'clicks' : crawl_df.iloc[[hit['corpus_id']]]['clicks']})  
                 
        rows_list.append([title, url, dict1])  
         
df2 = pd.DataFrame(rows_list)

0it [00:00, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1it [00:00,  9.10it/s]

ref: https://www.fila.com/men-heritage-outerwear
ref: https://www.fila.com/men-heritage-outerwear
recommended: https://www.fila.com/tennis-jackets-men
ref: https://www.fila.com/men-heritage-outerwear
recommended: https://www.fila.com/men-heritage-hoodies
ref: https://www.fila.com/men-heritage-outerwear
recommended: https://www.fila.com/men-heritage-tops
ref: https://www.fila.com/men-heritage-outerwear
recommended: https://www.fila.com/men-heritage-tracksuits


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

143it [00:17,  8.01it/s]

ref: https://www.fila.com/company-womens
recommended: https://www.fila.com/ohsu-womens
ref: https://www.fila.com/company-womens
ref: https://www.fila.com/company-womens
recommended: https://www.fila.com/cedars-women
ref: https://www.fila.com/company-womens
recommended: https://www.fila.com/women-sandals
ref: https://www.fila.com/company-womens
recommended: https://www.fila.com/women-heritage-tops


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

144it [00:17,  8.27it/s]

ref: https://www.fila.com/chunky-sneakers
ref: https://www.fila.com/chunky-sneakers
recommended: https://www.fila.com/shoes-men-raytracer
ref: https://www.fila.com/chunky-sneakers
recommended: https://www.fila.com/men-sneakers
ref: https://www.fila.com/chunky-sneakers
recommended: https://www.fila.com/tennis-sneakers-kids
ref: https://www.fila.com/chunky-sneakers
recommended: https://www.fila.com/shoes-men-sneakers


## 5. Preparing the output file

We are now preparing a json file that for each category page will provide title and url of the recommended links.

In [ ]:
df2 = df2[df2[2].str.len() > 0]
df3 = df2.join(pd.json_normalize(df2[2]))
df3 = df2.groupby(1)[2].apply(list).to_json('reccomended_links.json')

### Visualizing the recommendations

In [ ]:
def render(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)

# Opening the json file that we just created
f = open('/content/reccomended_links.json')
data = json.load(f)

# Rendering it in Colab
render(data)

## 6. Scaling the infrastructure

Here below we will re-run SBERT for embedding and similarity using the same dataset and [Jina AI NOW](https://now.jina.ai/). We do this test to demonstrate how to scale the workflow on a distributed enviroment. 

To do so we will install Jina AI. 


In [ ]:
!pip install jina

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
fila_df = pd.read_csv("/content/fila-com.csv")

#df = df.query("not clean_text.str.startswith('sitesfilasite fila.com sorry')")
fila_df = fila_df[fila_df.clean_text.apply(lambda x: len(x.split(" ")) > 4)]

fila_df.head(100)

,Unnamed: 0,url,title,clean_text
0,0,https://www.fila.com/men-featured,"FILA Men's Featured Shoes, Clothing, & Accesso...",fila mens featured shoes clothing accessory me...
1,1,https://www.fila.com/shoes-men,Men's Shoes,mens shoes collection mens shoes sneaker fila....
2,2,https://www.fila.com/men-sneakers,Men's Casual Sneakers + Athletic Shoes,mens casual sneakers athletic shoes casual sne...
3,3,https://www.fila.com/shoes-featured,Featured Shoes + Sneakers for Men + Women,featured shoe sneaker woman featured styles sh...
4,4,https://www.fila.com/men-slides,Men's Slides + Sport Sandals,mens slide sport sandal mens slide sport sanda...
...,...,...,...,...
95,95,https://www.fila.com/reilly-opelka,Reilly Opelka in FILA Tennis Apparel,reilly opelka fila tennis apparel reilly opelk...
96,96,https://www.fila.com/timea-babos,Tímea Babos in FILA Tennis Apparel,tmea babos fila tennis apparel tmea babos spor...
97,97,https://www.fila.com/tennis-women,Tennis Clothes for Women,tennis clothes woman fila tennis woman tennis ...
98,98,https://www.fila.com/company-mens,Men's 40% Off,fila.com fila small business mens mens small b...


In [ ]:
from docarray import DocumentArray

da = DocumentArray.from_dataframe(fila_df)

da.summary()

╭──────────── Documents Summary ────────────╮
│                                           │
│   Length                 98               │
│   Homogenous Documents   True             │
│   Common Attributes      ('id', 'tags')   │
│   Multimodal dataclass   False            │
│                                           │
╰───────────────────────────────────────────╯
╭───────────────────── Attributes Summary ─────────────────────╮
│                                                              │
│   Attribute   Data type   #Unique values   Has empty value   │
│  ──────────────────────────────────────────────────────────  │
│   id          ('str',)    98               False             │
│   tags        ('dict',)   98               False             │
│                                                              │
╰──────────────────────────────────────────────────────────────╯

In [ ]:
from docarray import DocumentArray
import requests

def jina_search(_query):

  data = {
    "host": "grpcs://nowapi-9e52d577e4.wolf.jina.ai",
    "text": _query,
    "limit": 5
  }

  response = requests.post('https://nowrun.jina.ai/api/v1/text-to-text/search', json=data)
  
  response = DocumentArray.from_json(response.content)

  result = response[:, ["tags__title", "tags__url", "scores__cosine__value"]]
  
  return result


In [ ]:
queries = ['boots for men', 'cycling', 'sneakers', 'kids']

In [ ]:
for query in queries:
  print(query, "\n", jina_search(query)[0],
        "\n", jina_search(query)[1],"\n",
        jina_search(query)[2])
